In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
import pandas as pd


os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.hadoop:hadoop-aws:3.2.2,io.delta:delta-core_2.12:1.1.0  pyspark-shell "

os.environ['MINIO_USERNAME'] = 'grupo-02'
os.environ['MINIO_PASSWORD'] = '6HgSzdwj8eNpHcux'
        
config = {
    "spark.jars.packages":"org.apache.hadoop:hadoop-aws:3.2.2",
    "spark.kubernetes.namespace": "spark",
    "spark.kubernetes.container.image": "cronosnull/abd-spark-base:202301",
    "spark.executor.instances": "17",
    "spark.executor.memory": "7g",
    "spark.executor.cores": "1",
    "spark.driver.memory":"7g",
    "spark.driver.port":"38889",
    "spark.driver.blockManager.port":"7777",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.driver.host": "172.24.99.147",
    "spark.kubernetes.executor.request.cores":"500m",
    "spark.hadoop.fs.s3a.endpoint": "http://172.24.99.18:9000",
    # Credenciales de MinNIO, no olvide asignar las variables de entorno  
    "spark.hadoop.fs.s3a.access.key": os.environ.get('MINIO_USERNAME', "--"),
    "spark.hadoop.fs.s3a.secret.key": os.environ.get('MINIO_PASSWORD', "--"),
    "spark.hadoop.fs.s3a.path.style.access": True,
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    "spark.kubernetes.local.dirs.tmpfs":True,

}

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://172.24.99.68:16443")
    for key, value in config.items():
        conf.set(key, value)    
        conf.set("spark.ui.port","4040")
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()


In [2]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_322
Branch 
Compiled by user  on 2022-03-26T09:34:47Z
Revision 
Url 
Type --help for more information.


In [3]:
spark = get_spark_session("grupo-02-app-prueba", SparkConf())
spark

In [ ]:

if spark is not None:
    print("La sesión de Spark se creó correctamente.")
else:
    print("Ocurrió un error al crear la sesión de Spark.")


# Parte I

## A (Title)

In [ ]:
from pyspark.sql.functions import *

# Lee el archivo JSON en un DataFrame de Spark
df = spark.read.format("json").option("header", "false").load("s3a://noticias2016/individual_files/*.json.gz")
df.createOrReplaceTempView("noticias")

##STOPWORDS
spStopWords = spark.read.text("s3a://user-data/grupo-02/stopwords/spanish")
spStopWords.createOrReplaceTempView("stopwords")

# Almacenar los datos en caché para mejorar el rendimiento de consultas repetidas
df = df.repartition(200, "title").cache()

# Ejecuta una consulta SQL en la vista temporal
result = spark.sql("""

  WITH DATA AS (SELECT lower(A.word) AS title , 1 veces
   FROM
     (SELECT EXPLODE(SPLIT(title, ' ')) AS word
      FROM noticias) A
   LEFT JOIN stopwords B ON lower(B.value) = lower(A.word)
   WHERE B.value IS NULL)
   
   SELECT  REGEXP_REPLACE(title, '[^0-9A-Za-z]' , '' ) title, sum(veces) veces
   FROM DATA A
   where REGEXP_REPLACE(title, '[^0-9A-Za-z]' , '' ) <>''
   group by REGEXP_REPLACE(title, '[^0-9A-Za-z]' , '' )
   order by 2 desc
   
   
""")

# Muestra las primeras 100 filas del resultado
result.show(10)

## B (Body)

In [ ]:
from pyspark.sql.functions import *

# Lee el archivo JSON en un DataFrame de Spark
df0 = spark.read.format("json").option("header", "false").load("s3a://noticias2016/individual_files/*.json.gz")
df0.createOrReplaceTempView("noticias")

##STOPWORDS
spStopWords = spark.read.text("s3a://user-data/grupo-02/stopwords/spanish")
spStopWords.createOrReplaceTempView("stopwords")

# Almacenar los datos en caché para mejorar el rendimiento de consultas repetidas
df0 = df0.repartition(200, "text").cache()

# Ejecuta una consulta SQL en la vista temporal
result = spark.sql("""

  WITH DATA AS (SELECT lower(A.text) AS text , 1 veces , b.value
   FROM
     (SELECT EXPLODE(SPLIT(text, ' ')) AS text
      FROM noticias) A
   LEFT JOIN stopwords B ON B.value = lower(A.text)
   where B.value is null
   
   )
   
   
   SELECT  
   REGEXP_REPLACE(a.text, '[^0-9A-Za-z]' , '' ) text, 
   sum(a.veces) veces
   
   FROM DATA A
   left join stopwords b on REGEXP_REPLACE(a.text, '[^0-9A-Za-z]' , '' ) = b.value
   
   where REGEXP_REPLACE(a.text, '[^0-9A-Za-z]' , '' ) <>''
   and B.value is null
   
   group by 
   REGEXP_REPLACE(a.text, '[^0-9A-Za-z]' , '' )
  
   order by 2 desc
   
   
""")

# Muestra las primeras 100 filas del resultado
result.show(10)

# Parte II

In [ ]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, when
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format


# Quitar espacios en los nombres es las columnas
def remove_space_in_names(df):
    for col_name in df.columns:
        new_col_name = col_name.replace(' ', '')
        df = df.withColumnRenamed(col_name, new_col_name)
    return df


# Cambiar los tipos de datos del dataframe
# Falta Rate_Code, store_and_forward, mta_tax
def change_name_and_data_type_G1(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("vendor_name", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("Trip_Pickup_DateTime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("Trip_Dropoff_DateTime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("Passenger_Count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("Trip_Distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("Start_Lon", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("Start_Lat", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("Rate_Code", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_forward", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("End_Lon", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("End_Lat", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("Payment_Type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("Fare_Amt", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("Tip_Amt", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("Tolls_Amt", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("Total_Amt", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df

def change_name_and_data_type_G2(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("vendor_id", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("pickup_longitude", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("pickup_latitude", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("rate_code", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("dropoff_longitude", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("dropoff_latitude", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
   
#     #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df


def change_name_and_data_type_G3(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("VendorID", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("pickup_longitude", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("pickup_latitude", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("RateCodeID", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("dropoff_longitude", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("dropoff_latitude", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("improvement_surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
   
    #Eliminar columna no usada
    df = df.drop("extra")
    
   
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df


def change_name_and_data_type_G4(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("VendorID", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("RateCodeID", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumn("PULocationID", col("PULocationID").cast("integer"))
    df = df.withColumn("PULocationID", col("PULocationID").cast("integer"))
    
#     df = df.withColumnRenamed("PULocationID", "pickup_zone").withColumn("pickup_zone", col("pickup_zone").cast("string"))
#     df = df.withColumnRenamed("PULocationID", "dropoff_zone").withColumn("pickup_zone", col("pickup_zone").cast("string"))

    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("improvement_surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
    #Eliminar columna no usada
    df = df.drop("extra")
    
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df

In [ ]:

# charge info group 1
df_taxis_G1 = spark.read.csv("s3a://taxis/yellow_tripdata_2009-*.csv.gz", header=True)
df_taxis_G1_change_type = change_name_and_data_type_G1(df_taxis_G1)

df_taxis_G1_change_type.printSchema()

In [ ]:
# charge info group 2

df_taxis_G2_2010 = spark.read.csv("s3a://taxis/yellow_tripdata_2010-*.csv.gz", header=True)
df_taxis_G2_2010_change_type = change_name_and_data_type_G2(df_taxis_G2_2010)

df_taxis_G2_2011 = spark.read.csv("s3a://taxis/yellow_tripdata_2011-*.csv.gz", header=True)
df_taxis_G2_2011_change_type = change_name_and_data_type_G2(df_taxis_G2_2011)

df_taxis_G2_2012 = spark.read.csv("s3a://taxis/yellow_tripdata_2012-*.csv.gz", header=True)
df_taxis_G2_2012_change_type = change_name_and_data_type_G2(df_taxis_G2_2012)

df_taxis_G2_2013 = spark.read.csv("s3a://taxis/yellow_tripdata_2013-*.csv.gz", header=True)
df_taxis_G2_2013_change_type = change_name_and_data_type_G2(df_taxis_G2_2013)

df_taxis_G2_2014 = spark.read.csv("s3a://taxis/yellow_tripdata_2014-*.csv.gz", header=True)
df_taxis_G2_2014_change_type = change_name_and_data_type_G2(df_taxis_G2_2014)


df_taxis_G2_change_type = df_taxis_G2_2010_change_type.unionAll(df_taxis_G2_2011_change_type).unionAll(df_taxis_G2_2013_change_type).unionAll(df_taxis_G2_2014_change_type)

df_taxis_G1_change_type.printSchema()


In [ ]:
# charge info group 3

df_taxis_G3_2015 = spark.read.csv("s3a://taxis/yellow_tripdata_2015-*.csv.gz", header=True)
df_taxis_G3_2015_change_type = change_name_and_data_type_G3(df_taxis_G3_2015)

df_taxis_G3_2016 = spark.read.csv("s3a://taxis/yellow_tripdata_2016-*.csv.gz", header=True)
df_taxis_G3_2016_change_type = change_name_and_data_type_G3(df_taxis_G3_2016)

df_taxis_G3_change_type = df_taxis_G3_2015_change_type.unionAll(df_taxis_G3_2016_change_type)

df_taxis_G3_change_type.printSchema()

In [ ]:
# charge info group 4

df_taxis_G4_2017 = spark.read.csv("s3a://taxis/yellow_tripdata_2017-*.csv.gz", header=True)
df_taxis_G4_2017_change_type = change_name_and_data_type_G4(df_taxis_G4_2017)

df_taxis_G4_2018 = spark.read.csv("s3a://taxis/yellow_tripdata_2018-*.csv.gz", header=True)
df_taxis_G4_2018_change_type = change_name_and_data_type_G4(df_taxis_G4_2018)

df_taxis_G4_2019 = spark.read.csv("s3a://taxis/yellow_tripdata_2019-*.csv.gz", header=True)
df_taxis_G4_2019 = df_taxis_G4_2019.drop("congestion_surcharge")
df_taxis_G4_2019_change_type = change_name_and_data_type_G4(df_taxis_G4_2019)


df_taxis_G4_change_type = df_taxis_G4_2017_change_type.unionAll(df_taxis_G4_2018_change_type).unionAll(df_taxis_G4_2019_change_type)

df_taxis_G4_change_type.printSchema()


In [ ]:
# Add the zone name
df_taxis_G4_change_type_2 = df_taxis_G4_change_type

df_location_id = spark.read.csv("s3a://user-data/grupo-02/taxi+_zone_lookup.csv", header=True)

df_location_id.head(4)

In [ ]:
# Add the zone to the dataframe 4 
df_taxis_G4_change_type_2 = df_taxis_G4_change_type

df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.join(df_location_id, df_taxis_G4_change_type_2.PULocationID == df_location_id.LocationID).select(df_taxis_G4_change_type_2['*'], df_location_id['Zone'])
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.withColumnRenamed("Zone", "pickup_zone")

df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.alias('a').join(df_location_id.alias('b'), df_taxis_G4_change_type_2.alias('a').DOLocationID == df_location_id.alias('b').LocationID).select('a.*','b.Zone')
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.withColumnRenamed("Zone", "dropoff_zone")
    
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.drop('DOLocationID','PULocationID')
df_taxis_G4_change_type_2.printSchema()


In [ ]:
# Join all the data with latitude and longitud

df_taxis_with_lat_lon = df_taxis_G1_change_type.unionAll(df_taxis_G2_change_type).unionAll(df_taxis_G3_change_type)

df_taxis_with_lat_lon.printSchema()

# geo pandas

In [ ]:
# !pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType,DecimalType
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf

import shapely.speedups
shapely.speedups.enable() # this makes some spatial queries run faster

#Load the data 
df_taxi_zones_csv = spark.read.csv("s3a://user-data/grupo-02/taxi_zones.csv", header=True)
df_taxi_zones_csv.printSchema()

# define a function to convert a WKT string to a Shapely geometry object
def wkt_to_geometry(wkt_str):
    return wkt.loads(wkt_str)

# read in the CSV file and convert the 'the_geom' column to a geometry data type
df_taxi_zones_csv = pd.read_csv("taxi_zones.csv", dtype={'the_geom': 'str'}).assign(the_geom=lambda x: x['the_geom'].apply(wkt_to_geometry))

# print the data types of the columns
print(df_taxi_zones_csv.dtypes)

gdf  = gpd.GeoDataFrame(df_taxi_zones_csv, geometry='the_geom')

# spark.broadcast(gdf)

def find_zone(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['zone'] if x['the_geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value



find_zone_udf = udf(lambda lat, lon: find_zone(lat, lon), StringType())




In [ ]:

df_taxis_with_lat_lon= df_taxis_with_lat_lon.withColumn("pickup_zone", find_zone_udf(col("pickup_latitude"),col("pickup_longitude")))

df_taxis_with_lat_lon= df_taxis_with_lat_lon.withColumn("dropoff_zone", find_zone_udf(col("dropoff_latitude"),col("dropoff_longitude")))

df_taxis_with_lat_lon = df_taxis_with_lat_lon.drop('pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude')
df_taxis_with_lat_lon.printSchema()


In [ ]:
# Join all the data

final_df_taxi_zones  = df_taxis_with_lat_lon.unionAll(df_taxis_G4_change_type_2)

final_df_taxi_zones= final_df_taxi_zones.withColumn("year_travel", date_format("pickup_datetime", "yyyy").alias("year_travel"))
final_df_taxi_zones= final_df_taxi_zones.withColumn("month_travel", date_format("pickup_datetime", "MM").alias("month_travel"))
final_df_taxi_zones= final_df_taxi_zones.withColumn("day_travel", date_format("pickup_datetime", "dd").alias("day_travel"))


final_df_taxi_zones = final_df_taxi_zones.withColumn("year_travel", col("year_travel").cast("integer"))
final_df_taxi_zones = final_df_taxi_zones.withColumn("month_travel", col("year_travel").cast("integer"))
final_df_taxi_zones = final_df_taxi_zones.withColumn("day_travel", col("year_travel").cast("integer"))


final_df_taxi_zones.printSchema()


In [ ]:
# Se crea un dataframe para analizar los datos

pd_df_final_df_taxi_zones = final_df_taxi_zones.toPandas()


In [ ]:
# Se sube el CSV a Minio

final_df_taxi_zones.write.mode("overwrite").csv("s3a://user-data/grupo-02/prueba/prueba.csv")


# Analisis de los datos 

## punto G

In [4]:
from pyspark.sql.functions import year

data = spark.read.format("csv").option("header", "true").load("s3a://user-data/grupo-02/final_df_taxi_zones_test.csv")

# Seleccionar columnas necesarias
data = data.select("pickup_zone", "dropoff_zone", "year_travel")

# Agrupar por año, zona de origen y zona de entrega y hacer el conteo
grouped_data = data.groupBy(year("year_travel").alias("year"), "pickup_zone", "dropoff_zone").count()

# Ordenar por año y conteo de manera descendente
sorted_data = grouped_data.orderBy("year", "count", ascending=[True, False])

# Mostrar los primeros 10 registros
sorted_data.show(10)

# Convertir a pandas y mostrar los primeros 5 registros
sorted_data.limit(5).toPandas().head(5)



+----+--------------------+--------------------+-----+
|year|         pickup_zone|        dropoff_zone|count|
+----+--------------------+--------------------+-----+
|2009|                null|                null|    4|
|2009|Upper East Side S...|Upper East Side S...|    3|
|2009|      Yorkville West|Upper East Side S...|    2|
|2009|Upper West Side N...|Upper West Side N...|    2|
|2009|Upper East Side S...|Upper East Side N...|    2|
|2009|      Midtown Center|      Midtown Center|    2|
|2009|Long Island City/...|Long Island City/...|    2|
|2009|Upper West Side S...|      Yorkville East|    2|
|2009|        Central Park| Lincoln Square East|    2|
|2009|      Midtown Center|       Midtown North|    2|
+----+--------------------+--------------------+-----+
only showing top 10 rows



,year,pickup_zone,dropoff_zone,count
0,2009,None,None,4
1,2009,Upper East Side South,Upper East Side South,3
2,2009,Yorkville West,Upper East Side South,2
3,2009,Midtown Center,Midtown Center,2
4,2009,Long Island City/Queens Plaza,Long Island City/Queens Plaza,2


## pivoteo de dropoff_zone

In [28]:
from pyspark.sql.functions import year

# Leer la tabla
data = spark.read.format("csv").option("header", "true").load("s3a://user-data/grupo-02/final_df_taxi_zones_test.csv")

# Seleccionar columnas necesarias y agrupar
grouped_data = data.select("pickup_zone", "dropoff_zone", "year_travel").groupBy(year("year_travel").alias("year"), "pickup_zone", "dropoff_zone").count()

# Generar la matriz
pivoted_data = grouped_data.groupBy("dropoff_zone").pivot("pickup_zone").sum("count")

# Ordenar la matriz y mostrar los primeros 10 registros
sorted_data = pivoted_data.orderBy("dropoff_zone", ascending=True).na.fill(0)
sorted_data.show(2)



+-------------+----+------------+-----------------+----------------+------------+--------------+------------+---------+------------+------------+------------+-----------------+------------+------------------------+--------+------------+----------------+--------+-----------------------+-----------------------+---------+-----------+--------+-----------------+---------------+---------------+-------------------+-------------------+-------------------+-----------------------------+---------------+-----------------------------+--------------+------------+-------------+-------------+-------------------+-----------+----------------------------+-----------------------+-----------------------------+-------------------------+--------------------+--------+---------------------+---------------------+---------------------+---------------------+-------------------------+------------+------------------+--------------+--------------+
| dropoff_zone|null|Battery Park|Battery Park City|Bensonhurst East|B

## pivoteo con pandas

In [7]:
from pyspark.sql.functions import year
import pandas as pd

data = spark.read.format("csv").option("header", "true").load("s3a://user-data/grupo-02/final_df_taxi_zones_test.csv")

# Seleccionar columnas necesarias
data = data.select("pickup_zone", "dropoff_zone", "year_travel")

# Agrupar por año, zona de origen y zona de entrega y hacer el conteo
grouped_data = data.groupBy(year("year_travel").alias("year"), "pickup_zone")\
                   .pivot("dropoff_zone")\
                   .count()

# Ordenar por año y zona de origen
sorted_data = grouped_data.orderBy("year", "pickup_zone")

# Mostrar los primeros 10 registros  ----sorted_data.show(10)

# Convertir a pandas y mostrar los primeros 5 registros
sorted_data.limit(5).toPandas().head(5)

# Convertir a pandas
pandas_data = sorted_data.toPandas()
pandas_data.iloc[:1, :1]

pandas_data = sorted_data.toPandas().fillna(value=0)


# Mostrar los primeros 5 registros
print(pandas_data.head(5))


   year        pickup_zone  null  Alphabet City  Astoria  Central Park  \
0  2009                  0   4.0            0.0      0.0           0.0   
1  2009       Battery Park   0.0            0.0      0.0           0.0   
2  2009  Battery Park City   0.0            0.0      0.0           0.0   
3  2009   Bensonhurst East   0.0            0.0      0.0           0.0   
4  2009       Bloomingdale   0.0            0.0      0.0           0.0   

   Clinton East  Clinton West  Coney Island  DUMBO/Vinegar Hill  ...  \
0           0.0           0.0           0.0                 0.0  ...   
1           0.0           0.0           0.0                 0.0  ...   
2           0.0           0.0           0.0                 1.0  ...   
3           0.0           0.0           0.0                 0.0  ...   
4           0.0           0.0           0.0                 0.0  ...   

   Union Sq  Upper East Side North  Upper East Side South  \
0       0.0                    0.0                    0.0   


## instalacion matplolib

In [ ]:
!pip install matplotlib

In [12]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import bokeh.plotting as bop
import altair as al
import plotnine as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

## grafica Matriz de calor

como la instalacion e importacion de matplotlib no fue posible por error en versionamiento no fue posible graficar la matriz de calor

In [5]:
import numpy as np
import matplotlib.pyplot as plt


# Leer la tabla
data = spark.read.format("csv").option("header", "true").load("s3a://user-data/grupo-02/final_df_taxi_zones_test.csv")


# seleccionar datos de interés
data = sorted_data.select("pickup_zone", "dropoff_zone", "year", "count")

# crear matriz pivot
matrix = data.groupBy("pickup_zone").pivot("dropoff_zone").sum("count").na.fill(0).orderBy("pickup_zone").toPandas().values[:,1:]

# crear figura y gráfico de matriz de calor
fig, ax = plt.subplots()
im = ax.imshow(matrix)

# ajustar etiquetas y títulos
ax.set_xticks(np.arange(len(sorted_data.select("dropoff_zone").distinct().collect())))
ax.set_yticks(np.arange(len(sorted_data.select("pickup_zone").distinct().collect())))
ax.set_xticklabels(sorted_data.select("dropoff_zone").distinct().rdd.map(lambda x: x[0]).collect())
ax.set_yticklabels(sorted_data.select("pickup_zone").distinct().rdd.map(lambda x: x[0]).collect())
ax.set_xlabel('dropoff_zone')
ax.set_ylabel('pickup_zone')
ax.set_title('Matriz de calor')

# agregar barras de color
cbar = ax.figure.colorbar(im, ax=ax)

# mostrar gráfico
plt.show()



ModuleNotFoundError: No module named 'matplotlib'

## punto H

In [ ]:
most_popular_destinations_per_years = pd_df_final_df_taxi_zones.groupby(['year_travel', 'dropoff_zone']).size().reset_index(name='count')

most_popular_destinations_per_years = most_popular_destinations_per_years.sort_values(['year_travel', 'count'], ascending=[False, False])

most_popular_destinations_per_years


## punto i

In [ ]:
df_km_per_year = pd_df_final_df_taxi_zones.groupby(['year_travel'])['trip_distance'].sum()

df_km_per_year = df_km_per_year.to_frame().reset_index()

df_km_per_year["trip_distance_km"] = df_km_per_year["trip_distance"] * 1.609344

In [ ]:
plt.plot(df_km_per_year["year_travel"], df_km_per_year["trip_distance_km"], '-o')


plt.xlabel('Año')
plt.xlim(2007, 2020)

plt.ylabel('Distancia total recorrida en Kilometros ')

plt.title(' Distancia total recorrida en Kilometros por año')

plt.show()

In [ ]:
spark.stop()